<a href="https://colab.research.google.com/github/shradhadabhade/AIES/blob/main/Pr5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install LIME

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for LIME: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=c2c553074c651cb3741be5af38e3d87191b9384722e9ec5a71b9def8e259def6
  Stored in directory: /root/.cache/pip/wheels/e7/5d/0e/4b4fff9a47468fed5633211fb3b76d1db43fe806a17fb7486a
Successfully built LIME


In [ ]:
# problem5_lime_breastcancer.py
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# LIME
try:
    from lime.lime_tabular import LimeTabularExplainer
except Exception as e:
    raise RuntimeError("Please install lime: pip install lime") from e

data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target
# reduce features to keep explanations readable
cols = ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness']
X = X[cols]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
clf = RandomForestClassifier(n_estimators=200, random_state=0).fit(X_train, y_train)

# choose an instance
inst_idx = 3
x = X_test.iloc[inst_idx].values

explainer = LimeTabularExplainer(
    training_data=X_train.values,
    feature_names=cols,
    class_names=['malignant', 'benign'],
    mode='classification',
    discretize_continuous=True
)
exp = explainer.explain_instance(x, clf.predict_proba, num_features=5)
print("Predicted probabilities:", clf.predict_proba([x])[0])
print("\nLIME explanation (feature, weight):")
for feat, weight in exp.as_list():
    print(feat, weight)


Predicted probabilities: [0.08 0.92]

LIME explanation (feature, weight):
mean smoothness <= 0.09 0.14931811791998623
86.29 < mean perimeter <= 103.78 -0.07413968269198254
16.17 < mean texture <= 18.81 0.04440544131602675
13.38 < mean radius <= 15.75 0.026943517204523815
552.60 < mean area <= 771.77 0.013059721906624118


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


**Explanation**:

**Prediction Interpretation**

The classifier predicted Class 1 (Benign) with probability 0.92, meaning the model is very confident that this tumor is benign.

**Purpose of LIME**

LIME explains why the model made this prediction for this specific patient/sample.

It does this by approximating the model locally with an interpretable linear model.

**Understanding the Explanation Table**

| Feature Condition                         | Weight         | Interpretation                                                                   |
| ----------------------------------------- | -------------- | -------------------------------------------------------------------------------- |
| **mean smoothness ≤ 0.09**                | **+0.149**     | Low smoothness strongly supports the tumor being **benign**.                     |
| **86.29 < mean perimeter ≤ 103.78**       | **−0.074**     | Larger perimeter suggests the tumor could be **malignant**, but only moderately. |
| **mean texture, radius, and area ranges** | small +weights | These slightly reinforce the **benign classification**, but with lower impact.   |



Positive weight → pushes prediction toward Benign


Negative weight → pushes prediction toward Malignant


Conclusion

“LIME shows that the most influential factor in classifying this tumor as benign is its low mean smoothness value. Some features like perimeter slightly push the prediction toward malignant, but not strongly enough to change the final decision. The final predicted probability of 0.92 aligns well with these contributing feature effects, giving a clear and interpretable explanation for the model’s confidence.”


# Question	Expected Answer
1. What does the predicted probability [0.08 0.92] mean?

	The model predicts a 92% probability for the positive class (likely malignant tumor) and 8% for negative (benign). So the predicted class is the positive class.
2. What is LIME?

	LIME (Local Interpretable Model-agnostic Explanations) explains the contribution of features for one specific prediction rather than the whole model.
3. Why do we call LIME “local interpretability”?

Because it interprets a single instance (one patient or one record), not the entire dataset.
4. What does the positive weight of mean smoothness <= 0.09 indicate?

	It supports the predicted class (malignant). A positive weight means the feature pushes the model towards the predicted output.
5. What does the negative weight for mean perimeter indicate?

It means this feature pulls the prediction away from the predicted class. It decreases confidence in the malignant prediction.
6. Why are decision rules shown as ranges (e.g., 13.38 < radius <= 15.75)?

	LIME discretizes continuous features to make explanations human-readable.
7. Why do we use LIME instead of just looking at prediction probability?

Probability tells what the model predicted; LIME explains why it predicted that.
8. LIME gave a warning about feature names — does this affect interpretation?

	No. The warning indicates that input features were passed without column names, but the interpretation remains correct.
9. Why is LIME useful in healthcare?

Doctors need transparent reasoning before relying on AI predictions. LIME supports clinical decision-making with feature-based explanations.